# 模块1：环境配置与数据集定义

In [1]:
# 模块1：数据集和数据加载相关代码

import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 检查GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, transform=None, partition=0):
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        attr_df = pd.read_csv(attr_path, sep=',', header=0)
        partition_df = pd.read_csv(partition_path, sep=',', header=0)
        attr_df = attr_df.merge(partition_df, on='image_id')
        self.attr_df = attr_df[attr_df['partition'] == partition]

        bbox_df = pd.read_csv(bbox_path, sep=',', header=0)
        self.attr_df = self.attr_df.merge(bbox_df, on='image_id')

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        img_name = self.attr_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        attrs = self.attr_df.iloc[idx, 1:41].values
        attrs = (attrs + 1) // 2
        attrs = attrs.astype(np.float32)
        
        if self.transform:
            image = self.transform(image)
        
        return image, attrs

# 定义图像预处理
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # 对RGB三通道使用相同的归一化
])

# 数据集路径设置（请根据实际路径修改）
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'


使用设备: cuda


# 模块2：加载数据集

In [2]:
# 模块2：创建数据加载器

def get_dataloaders(batch_size=128, num_workers=4):
    train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, transform=transform, partition=0)
    val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, transform=transform, partition=1)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader


# 模块3：定义CVAE模型

In [3]:
# 模块3：定义CVAE模型

class CVAE(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, cond_dim=40):
        super(CVAE, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim

        # 编码器
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim, 64, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器
        self.decoder_input = nn.Linear(latent_dim + cond_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, 4, 2, 1),
            nn.Tanh()
        )

    def encode(self, x, c):
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c):
        z = torch.cat([z, c], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c):
        mu, logvar = self.encode(x, c)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c)
        return recon_x, mu, logvar


# 模块4：定义损失函数与训练函数

In [4]:
# 模块4：训练与验证函数以及损失函数定义

criterion = nn.MSELoss(reduction='sum')  # 重建损失使用MSE
def loss_function(recon_x, x, mu, logvar):
    recon_loss = criterion(recon_x, x)
    KL = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + KL

def train_one_epoch(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    for data, attrs in train_loader:
        data = data.to(device)
        attrs = attrs.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data, attrs)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader.dataset)
    return avg_loss

def validate(model, val_loader, device):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, attrs in val_loader:
            data = data.to(device)
            attrs = attrs.to(device)
            recon_batch, mu, logvar = model(data, attrs)
            loss = loss_function(recon_batch, data, mu, logvar)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader.dataset)
    return avg_val_loss


# 模块5：单次训练流程与结果可视化

In [5]:
# 模块5：单次训练流程
# 您可以调用此模块进行一次训练过程（固定epoch次数），并返回训练与验证损失列表。

def train_model(epochs=20, latent_dim=128, cond_dim=40, learning_rate=1e-3):
    model = CVAE(latent_dim=latent_dim, cond_dim=cond_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_loader, val_loader = get_dataloaders()

    train_losses = []
    val_losses = []

    for epoch in range(1, epochs+1):
        train_loss = train_one_epoch(model, train_loader, optimizer, device)
        val_loss = validate(model, val_loader, device)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    return model, train_losses, val_losses

# 可视化损失曲线的函数
def plot_losses(train_losses, val_losses, title="Training Curve"):
    plt.figure(figsize=(8,6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(title)
    plt.legend()
    plt.show()


# 模块6：生成图像和属性修改

In [10]:
# 模块6：图像生成与展示函数

def generate_images(model, attrs, device, latent_dim=128, num_images=16):
    model.eval()
    with torch.no_grad():
        z = torch.randn(num_images, latent_dim).to(device)
        attrs = attrs.to(device)
        generated = model.decode(z, attrs)
        generated = generated.cpu()
        return generated
    
def show_images(images, title="Generated Images", save_path=None):
    images = images * 0.5 + 0.5  # 反归一化
    grid = torchvision.utils.make_grid(images, nrow=4)
    np_grid = grid.numpy()
    plt.figure(figsize=(8,8))
    plt.imshow(np.transpose(np_grid, (1, 2, 0)))
    plt.title(title)
    plt.axis('off')
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

# def show_images(images, title="Generated Images"):
#     images = images * 0.5 + 0.5  # 反归一化
#     grid = torchvision.utils.make_grid(images, nrow=4)
#     np_grid = grid.numpy()
#     plt.figure(figsize=(8,8))
#     plt.imshow(np.transpose(np_grid, (1, 2, 0)))
#     plt.title(title)
#     plt.axis('off')
#     plt.show()


# 模块7：单次实验示例与结果展示

In [9]:
import matplotlib
matplotlib.use('Agg')  # 使用非交互式后端

In [11]:
# 模块7：运行一次训练并可视化结果

model, train_losses, val_losses = train_model(epochs=20)
plot_losses(train_losses, val_losses, title="CVAE Training with 20 epochs")

# 从验证集拿一个batch演示生成
train_loader, val_loader = get_dataloaders()
images, attrs = next(iter(val_loader))
sample_attrs = attrs[:16]
generated_images = generate_images(model, sample_attrs, device=device, num_images=16)
show_images(generated_images, title="Sample Generated Images", save_path="generated_sample.png")


# 修改某个属性（如Smiling）
modified_attrs = sample_attrs.clone()
modified_attrs[:, 31] = 1  # 假设第32列为Smiling属性
modified_generated = generate_images(model, modified_attrs, device=device, num_images=16)
show_images(modified_generated, title="Modified 'Smiling' Attribute Images")


Epoch 1, Train Loss: 389729380663.9202, Val Loss: 1441.0903
Epoch 2, Train Loss: 1228.4961, Val Loss: 1218.4309
Epoch 3, Train Loss: 1084.6629, Val Loss: 1082.5778
Epoch 4, Train Loss: 991.3360, Val Loss: 930.8724
Epoch 5, Train Loss: 936.0570, Val Loss: 964.3071
Epoch 6, Train Loss: 902.4863, Val Loss: 872.2933
Epoch 7, Train Loss: 866.2246, Val Loss: 899.5244
Epoch 8, Train Loss: 844.9742, Val Loss: 849.2684
Epoch 9, Train Loss: 825.8536, Val Loss: 847.0976
Epoch 10, Train Loss: 808.1083, Val Loss: 878.3733
Epoch 11, Train Loss: 801.5370, Val Loss: 819.8101
Epoch 12, Train Loss: 784.7714, Val Loss: 803.3191
Epoch 13, Train Loss: 780.1961, Val Loss: 779.1673
Epoch 14, Train Loss: 769.2927, Val Loss: 804.9587
Epoch 15, Train Loss: 765.6936, Val Loss: 779.0623
Epoch 16, Train Loss: 755.1654, Val Loss: 774.3531
Epoch 17, Train Loss: 789.8601, Val Loss: 827.2210
Epoch 18, Train Loss: 755.3821, Val Loss: 743.2994
Epoch 19, Train Loss: 757.8143, Val Loss: 800.2900
Epoch 20, Train Loss: 738.

# 模块8：多次训练求平均收敛点
下面的代码是回答您的第一点需求：相同轮次下，多次训练取得平均收敛曲线。比如您想在固定的 epochs=20 下，运行5次训练，然后计算这5次训练在各个epoch的平均损失和验证损失。

In [8]:
# 模块8：多次实验以求平均收敛点
def multiple_runs(epochs=20, runs=5):
    all_train_losses = []
    all_val_losses = []

    for run in range(runs):
        print(f"Run {run+1}/{runs}")
        _, train_losses, val_losses = train_model(epochs=epochs)
        all_train_losses.append(train_losses)
        all_val_losses.append(val_losses)

    # 将多次训练的损失求平均
    avg_train_losses = np.mean(all_train_losses, axis=0)
    avg_val_losses = np.mean(all_val_losses, axis=0)
    return avg_train_losses, avg_val_losses

# 示例：运行5次，并画出平均收敛曲线
avg_train_losses, avg_val_losses = multiple_runs(epochs=20, runs=5)
plot_losses(avg_train_losses, avg_val_losses, title="Average Curve over 5 runs (20 epochs)")


Run 1/5
Epoch 1, Train Loss: 343044610082.1857, Val Loss: 1313.6351
Epoch 2, Train Loss: 1222.3490, Val Loss: 1145.7170
Epoch 3, Train Loss: 1043.6424, Val Loss: 984.0562
Epoch 4, Train Loss: 942.9426, Val Loss: 912.9837
Epoch 5, Train Loss: 885.9244, Val Loss: 931.4077
Epoch 6, Train Loss: 854.1712, Val Loss: 857.8686
Epoch 7, Train Loss: 831.7365, Val Loss: 894.2481


KeyboardInterrupt: 

# 模块9：改变训练轮次进行多组实验
接下来是第二点需求：在完成多次相同轮次训练后，我们还希望改变轮数，比如尝试 epochs = 10, 20, 30, 40，每个轮数都进行多次训练，以比较平均收敛点和验证性能。这可以在一个循环中进行。

In [ ]:
# 模块9：改变轮次并多次训练对比
epoch_list = [10, 20, 30, 40]
runs_per_setting = 3  # 每个轮次数多次训练的次数

results = {}
for ep in epoch_list:
    print(f"==== Testing {ep} epochs ====")
    avg_train_losses, avg_val_losses = multiple_runs(epochs=ep, runs=runs_per_setting)
    results[ep] = (avg_train_losses, avg_val_losses)

# 对比不同epoch设置的最终验证损失或曲线
plt.figure(figsize=(8,6))
for ep in epoch_list:
    avg_train, avg_val = results[ep]
    plt.plot(avg_val, label=f'{ep} epochs')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.title('Validation Loss Comparison for Different Epoch Counts')
plt.legend()
plt.show()
